# Ausgleichsenergiepreisprognose
Dieses Notebook sammelt Daten und entwickelt ein erstes Basismodell zur Vorhersage der Schweizer Ausgleichsenergiepreise.

## Datenbeschaffung
Die historischen positiven und negativen Ausgleichsenergiepreise werden monatlich von Swissgrid als Excel-Dateien bereitgestellt. Im Folgenden wird gezeigt, wie diese automatisch heruntergeladen und eingelesen werden können. Bei fehlender Internetverbindung wird ein Platzhalter erzeugt.

In [ ]:

import pandas as pd
from pathlib import Path
import requests

BASE_URL = "https://www.swissgrid.ch/dam/swissgrid/current/Data/excel/balancingenergy/"

def download_monthly_price(year: int, month: int, folder: Path = Path('data')) -> Path:
    folder.mkdir(exist_ok=True)
    filename = folder / f"{year}_{month:02d}_balancing_energy_prices.xlsx"
    url = f"{BASE_URL}{year}/{month:02d}/balancing_energy_prices_{year}_{month:02d}.xlsx"
    try:
        r = requests.get(url)
        r.raise_for_status()
        filename.write_bytes(r.content)
        print(f"Downloaded {filename}")
    except Exception as e:
        print(f"Could not download {url}: {e}
Generating fake data instead.")
        dummy = pd.DataFrame({
            'Time': pd.date_range(f"{year}-{month:02d}-01", periods=96, freq='15min'),
            'PositivePrice': 0.0,
            'NegativePrice': 0.0
        })
        dummy.to_excel(filename, index=False)
    return filename


Nach dem Herunterladen werden die Daten in einen DataFrame geladen:

In [ ]:

def load_price_file(path: Path) -> pd.DataFrame:
    df = pd.read_excel(path)
    df = df.rename(columns=str.strip)
    df['Time'] = pd.to_datetime(df['Time'])
    return df.set_index('Time')

# Beispiel: Preisdatei fuer Januar 2024 herunterladen und einlesen
file_path = download_monthly_price(2024, 1)
prices = load_price_file(file_path)
prices.head()


### Zusätzliche Datenquellen
Um weitere Kontextinformationen einzubeziehen, können ENTSO-E Lastdaten, Wetterdaten und Schweizer Feiertage geladen werden. Bei fehlenden Bibliotheken oder Token werden Dummy-Daten erzeugt.

In [ ]:

from datetime import datetime, timedelta
import os

try:
    from entsoe import EntsoePandasClient
except ImportError:
    EntsoePandasClient = None

try:
    import holidays
except ImportError:
    holidays = None

# ENTSO-E load data

def fetch_entsoe_load(start: datetime, end: datetime, token: str) -> pd.DataFrame:
    if EntsoePandasClient is None:
        raise RuntimeError('entsoe-py not installed')
    client = EntsoePandasClient(api_key=token)
    try:
        load = client.query_load(country_code='CH', start=start, end=end)
        load = load.rename('load')
        return load.to_frame()
    except Exception as e:
        print(f'Could not download ENTSO-E data: {e}. Returning dummy data')
        idx = pd.date_range(start, end, freq='H', inclusive='left')
        return pd.DataFrame({'load': 0.0}, index=idx)

# MeteoSchweiz (Open Meteo) weather data

def fetch_weather(start: datetime, end: datetime, lat: float, lon: float) -> pd.DataFrame:
    url = (
        'https://api.open-meteo.com/v1/forecast'
        f'?latitude={lat}&longitude={lon}&hourly=temperature_2m,wind_speed_10m'
        f'&start_date={start.date()}&end_date={end.date()}&timezone=UTC'
    )
    try:
        r = requests.get(url)
        r.raise_for_status()
        data = r.json()
        times = pd.to_datetime(data['hourly']['time'])
        df = pd.DataFrame({'temperature': data['hourly']['temperature_2m'],
                           'wind_speed': data['hourly']['wind_speed_10m']},
                          index=times)
        return df
    except Exception as e:
        print(f'Could not download weather data: {e}. Returning dummy data')
        idx = pd.date_range(start, end, freq='H', inclusive='left')
        return pd.DataFrame({'temperature': 0.0, 'wind_speed': 0.0}, index=idx)

# Swiss holidays

def fetch_holidays(start: datetime, end: datetime) -> pd.DataFrame:
    if holidays is None:
        raise RuntimeError('holidays library not installed')
    ch_holidays = holidays.CH(years=range(start.year, end.year + 1))
    idx = pd.date_range(start, end, freq='D')
    flags = [1 if day.date() in ch_holidays else 0 for day in idx]
    return pd.DataFrame({'holiday': flags}, index=idx)


In [ ]:

start = prices.index.min()
end = prices.index.max() + pd.Timedelta(hours=1)

token = os.getenv('ENTSOE_TOKEN', '')
if token:
    load = fetch_entsoe_load(start, end, token)
else:
    print('ENTSOE_TOKEN not set. Using dummy load data')
    load = fetch_entsoe_load(start, end, token)

weather = fetch_weather(start, end, lat=46.8, lon=8.3)
holidays_df = fetch_holidays(start, end)

# Zusammenführen auf Stundenbasis
prices_h = prices.resample('H').mean()
data = prices_h.join(load, how='left').join(weather, how='left')
data = data.join(holidays_df, how='left')
data.head()


## Feature Engineering
Im Leitfaden sind zahlreiche potentielle Prädiktoren genannt. Für den Start erzeugen wir einfache Zeitfeatures und Lag-Variablen.

In [ ]:

def add_basic_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['hour'] = df.index.hour
    df['weekday'] = df.index.weekday
    df['month'] = df.index.month
    df['dayofyear'] = df.index.dayofyear
    df['lag_price'] = df['PositivePrice'].shift(1)
    df['rolling_mean_24h'] = df['PositivePrice'].rolling(window=24).mean()
    df['rolling_std_24h'] = df['PositivePrice'].rolling(window=24).std()
    return df

features = add_basic_features(prices)
features.dropna().head()


## Modellansatz
Für ein Minimalbeispiel verwenden wir ein Gradient Boosting Regressor auf Stundenbasis. Das Ziel ist der PositivePrice.

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor

X = features[['hour', 'weekday', 'month', 'dayofyear', 'lag_price', 'rolling_mean_24h', 'rolling_std_24h']].dropna()
y = features.loc[X.index, 'PositivePrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
model = GradientBoostingRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('MAE:', mean_absolute_error(y_test, y_pred))


Das Notebook stellt damit eine erste Vorlage dar, die nach Verfügbarkeit echter Daten weiter ausgebaut werden kann.